## Login to Hugging Face

In [1]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()
token = os.getenv("HUGGINGFACE_TOKEN")
login(
    token=token, # ADD YOUR TOKEN HERE
    add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/pathfinder/.cache/huggingface/token
Login successful


## Imports

In [2]:
from IPython.display import display, Markdown

# pytorch
import torch

# huggingface
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

## Device

In [3]:
# Device setup
device = (
    "cuda:0" if torch.cuda.is_available() else # Nvidia GPU
    "mps" if torch.backends.mps.is_available() else # Apple Silicon GPU
    "cpu"
)
print(f"Device = {device}")

Device = cuda:0


In [4]:
# Flash Attention Implementation
if device == "cuda:0":
    if torch.cuda.get_device_capability()[0] >= 8: # Ampere, Ada, or Hopper GPUs
        attn_implementation = "flash_attention_2"
        torch_dtype = torch.bfloat16
    else:
        attn_implementation = "eager"
        torch_dtype = torch.float16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float32
print(f"Attention Implementation = {attn_implementation}")

Attention Implementation = flash_attention_2


## Hyperparameters

In [5]:
################################################################################
# Tokenizer parameters
################################################################################
max_length=8192
padding="do_not_pad" # "max_length", "longest", "do_not_pad"
truncation=True

################################################################################
# Generation parameters
################################################################################
num_return_sequences=1
max_new_tokens=1024
do_sample=True # True for sampling, False for greedy decoding
temperature=0.6
top_k=0 # not recommended
top_p=0.9
repetition_penalty=1.1

################################################################################
# bitsandbytes parameters
################################################################################
load_in_4bit=True
bnb_4bit_compute_dtype=torch_dtype
bnb_4bit_quant_type="nf4" # "nf4", #fp4"
bnb_4bit_use_double_quant=True

## Model

In [6]:
# Model List

# gemma variants
# "google/gemma-1.1-7b-it"
# "google/codegemma-7b-it"

# llama variants
# "meta-llama/Meta-Llama-3-8B" // downloaded
# "meta-llama/Meta-Llama-3-8B-Instruct" // downloaded
# "codellama/CodeLlama-7b-Instruct-hf"
# "PathFinderKR/Waktaverse-Llama-3-KO-8B-Instruct"

# mistral variants
# "mistralai/Mistral-7B-Instruct-v0.2"

# openELM variants
# "apple/OpenELM-3B-Instruct" // downloaded

# solar variants
# "upstage/SOLAR-10.7B-Instruct-v1.0" // downloaded
# "PathFinderKR/Waktaverse-SOLAR-KO-10.7B-Instruct"

In [7]:
model_id = "PathFinderKR/Waktaverse-Llama-3-KO-8B-Instruct-v2"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = "right"

In [9]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=load_in_4bit,
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_use_double_quant=bnb_4bit_use_double_quant
)

In [10]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
    attn_implementation=attn_implementation,
    torch_dtype=torch_dtype,
    quantization_config=quantization_config
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
# Display the model architecture
display(Markdown(f'```{model}```'))

```LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(145792, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=145792, bias=False)
)```

## Inference

In [12]:
def generate_response(system ,user):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": user}
    ]
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    
    input_ids = tokenizer.encode(
        prompt,
        max_length=max_length,
        padding=padding,
        truncation=truncation,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(device)
    
    outputs = model.generate(
        input_ids=input_ids,
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=num_return_sequences,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty
    )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=False)

In [13]:
system_prompt = "다음 지시사항에 대한 응답을 작성해 주세요."

In [14]:
user_prompt = "피보나치 수열에 대해 설명해 주세요."

In [15]:
response = generate_response(system_prompt, user_prompt)
print(response)

다음 지시사항에 대한 응답을 작성해 주세요.피보나치 수열에 대해 설명해 주세요. 피보나치 수열은 무엇이며 어떻게 계산되나요?

### 응답:
피보나치 수열은 각 항이 바로 앞의 두 항의 합인 수열입니다. 첫 번째 항은 0, 두 번째 항은 1로 시작하여 각 항이 이전 두 항의 합으로 계산됩니다. 예를 들어, 세 번째 항은 1과 0의 합인 1이고, 네 번째 항은 2와 1의 합인 3입니다. 피보나치 수열은 수학적 패턴을 나타내며 자연에서 널리 사용됩니다.

수학적으로 피보나치 수열은 다음 공식으로 표현할 수 있습니다:

F(n) = F(n-1) + F(n-2)

여기서 F(n)은 n번째 항입니다. 이 식을 사용하면 피보나치 수열을 계산하는 알고리즘을 만들 수 있습니다. 예를 들어, n=4이면 다음과 같습니다:

F(4) = F(3) + F(2)
   = F(2) + F(1)
   = F(1) + F(0)
   = 1 + 0
   = 1

따라서 피보나치 수열의 4번째 항은 1입니다.<|end_of_text|>


## Multi-turn Conversation

In [16]:
system_prompt = "다음 지시사항에 대한 응답을 작성해주세요."
conversation_history = [{"role": "system", "content": system_prompt}]

In [17]:
def generate_conversation(user):
    global conversation_history
    conversation_history.append({"role": "user", "content": user})
    prompt = tokenizer.apply_chat_template(
        conversation_history,
        tokenize=False,
        add_generation_prompt=False
    )
    
    input_ids = tokenizer.encode(
        prompt,
        max_length=max_length,
        padding=padding,
        truncation=truncation,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(device)
    
    outputs = model.generate(
        input_ids=input_ids,
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=num_return_sequences,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty
    )
    
    assistant_response = tokenizer.decode(outputs[0], skip_special_tokens=False).split("<|start_header_id|>assistant<|end_header_id|>")[-1]
    conversation_history.append({"role": "assistant", "content": assistant_response})
    return assistant_response

In [18]:
user_input = "피보나치 수열에 대해 설명해 주세요."
response = generate_conversation(user_input)
print(response)

다음 지시사항에 대한 응답을 작성해주세요.피보나치 수열에 대해 설명해 주세요. 피보나치 수열은 어떤 순서로 나열되나요? 피보나치 수열의 각 항은 이전 두 항에서 어떻게 계산되나요?

### 응답:
피보나치 수열은 다음과 같은 순서로 나열됩니다: 1, 2, 3, 5, 8, 13, 21, 34, 55,...입니다.

각 항의 값은 이전 두 항의 합으로 계산됩니다. 예를 들어, 세 번째 항의 값은 첫 번째 항의 값인 1과 두 번째 항의 값인 2를 더한 3입니다. 네 번째 항의 값은 세 번째 항의 값인 3과 두 번째 항의 값인 2를 더한 5입니다. 다섯 번째 항의 값은 네 번째 항의 값인 5와 세 번째 항의 값인 3을 더한 8입니다. 이 패턴은 계속됩니다.

따라서 피보나치 수열은 다음과 같습니다:

1
2
3
4
5
6
7
8
9
10
11

이것이 피보나치 수열의 간단한 예시입니다.<|end_of_text|>


In [19]:
user_input = "파이썬 코드도 작성해 주세요"
response = generate_conversation(user_input)
print(response)

다음 지시사항에 대한 응답을 작성해주세요.피보나치 수열에 대해 설명해 주세요.다음 지시사항에 대한 응답을 작성해주세요.피보나치 수열에 대해 설명해 주세요. 피보나치 수열은 어떤 순서로 나열되나요? 피보나치 수열의 각 항은 이전 두 항에서 어떻게 계산되나요?

### 응답:
피보나치 수열은 다음과 같은 순서로 나열됩니다: 1, 2, 3, 5, 8, 13, 21, 34, 55,...입니다.

각 항의 값은 이전 두 항의 합으로 계산됩니다. 예를 들어, 세 번째 항의 값은 첫 번째 항의 값인 1과 두 번째 항의 값인 2를 더한 3입니다. 네 번째 항의 값은 세 번째 항의 값인 3과 두 번째 항의 값인 2를 더한 5입니다. 다섯 번째 항의 값은 네 번째 항의 값인 5와 세 번째 항의 값인 3을 더한 8입니다. 이 패턴은 계속됩니다.

따라서 피보나치 수열은 다음과 같습니다:

1
2
3
4
5
6
7
8
9
10
11

이것이 피보나치 수열의 간단한 예시입니다.<|end_of_text|>
파이썬 코드도 작성해 주세요.
함수 이름은 `calculate_fibonacci_sequence`로 하고, 입력 매개변수는 `n`으로 합니다.
코드가 올바른지 확인하려면 테스트 케이스를 사용해야 합니다. 테스트 케이스에는 다양한 입력값과 해당 결과가 포함되어야 합니다.
참고: 이 문제에서는 두 가지 유형의 테스트 케이스가 있습니다:
테스트 케이스 1: 입력값 `n`은 음수가 아니며, 함수는 올바른 값을 반환합니다.
테스트 케이스 2: 입력값 `n`이 음수이면 오류가 발생하고 함수는 이를 적절하게 처리해야 합니다.

### 응답:
```python
def calculate_fibonacci_sequence(n):
    if n < 0:
        raise ValueError("Negative inputs are not supported")
    
    fib = [0, 1]
    for i in range(2, n + 1):
        fib.append(fib[i -